# 1_뉴스 데이터 수집

## 1) 네이버 뉴스

In [1]:
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import re

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
#각 크롤링 결과 저장하기 위한 리스트 선언
title_text = []
link_text = []
source_text = []
date_text = []
contents_text = []
result = {}

#엑셀로 저장하기 위한 변수
RESULT_PATH ='/content/gdrive/MyDrive/통계분석실습'  #결과 저장할 경로
now = datetime.now() #파일이름 현 시간으로 저장하기

#날짜 정제화 함수
def date_cleansing(test):
    try:
        #지난 뉴스
        pattern = '\d+.(\d+).(\d+).'  #정규표현식

        r = re.compile(pattern)
        match = r.search(test).group(0)  # 2024.01.01
        date_text.append(match)

    except AttributeError:
        #최근 뉴스
        pattern = '\w* (\d\w*)'     #정규표현식

        r = re.compile(pattern)
        match = r.search(test).group(1)
        #print(match)
        date_text.append(match)

#내용 정제화 함수
def contents_cleansing(contents):
    first_cleansing_contents = re.sub('<dl>.*?</a> </div> </dd> <dd>', '',
                                      str(contents)).strip()  #앞에 필요없는 부분 제거
    second_cleansing_contents = re.sub('<ul class="relation_lst">.*?</dd>', '',
                                       first_cleansing_contents).strip() #뒤에 필요없는 부분 제거 (새끼 기사)
    third_cleansing_contents = re.sub('<.+?>', '', second_cleansing_contents).strip()
    contents_text.append(third_cleansing_contents)
    #print(contents_text)

def crawler(maxpage,query,sort,s_date,e_date):
    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    page = 1
    maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지

    while page <= maxpage_t:
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)

        response = requests.get(url)
        html = response.text

        #뷰티풀소프의 인자값 지정
        soup = BeautifulSoup(html, 'html.parser')

        #<a>태그에서 제목과 링크주소 추출
        atags = soup.select('.news_tit')
        for atag in atags:
            title_text.append(atag.text)     #제목
            link_text.append(atag['href'])   #링크주소

        #신문사 추출
        source_lists = soup.select('.info_group > .press')
        for source_list in source_lists:
            source_text.append(source_list.text)    #신문사

        #날짜 추출
        date_lists = soup.select('.info_group > span.info')

        for date_list in date_lists:
            # 1면 3단 같은 위치 제거
            if date_list.text.find("면") == -1:
                date_text.append(date_list.text)

        #본문요약본
        contents_lists = soup.select('.news_dsc')
        for contents_list in contents_lists:
            contents_cleansing(contents_list) #본문요약 정제화

        #모든 리스트 딕셔너리형태로 저장
        result= {"date" : date_text , "title":title_text ,  "source" : source_text ,"contents": contents_text ,"link":link_text }
        print(page)

        df = pd.DataFrame(result)  #df로 변환
        page += 10            # 새로 만들 파일이름 지정

    # 새로 만들 파일이름 지정
    outputFileName = '/2301.csv'
    df.to_csv(RESULT_PATH+outputFileName,index=False)

def main():
    info_main = input("="*50+"\n"+"입력 형식에 맞게 입력해주세요."+"\n"+" 시작하시려면 Enter를 눌러주세요."+"\n"+"="*50)
    maxpage = input("최대 크롤링할 페이지 수 입력하시오: ")
    query = input("검색어 입력: ")
    sort = input("뉴스 검색 방식 입력(관련도순=0  최신순=1  오래된순=2): ")    #관련도순=0  최신순=1  오래된순=2
    s_date = input("시작날짜 입력(2024.01.01):")  #2024.01.01
    e_date = input("끝날짜 입력(2024.02.01):")   #2024.02.01
    crawler(maxpage,query,sort,s_date,e_date)

main()

입력 형식에 맞게 입력해주세요.
 시작하시려면 Enter를 눌러주세요.
최대 크롤링할 페이지 수 입력하시오: 10
검색어 입력: 현대 전기차
뉴스 검색 방식 입력(관련도순=0  최신순=1  오래된순=2): 0
시작날짜 입력(2024.01.01):2023.01.01
끝날짜 입력(2024.02.01):2023.01.31
1
11
21
31
41
51
61
71
81
91


## 2) 해외 관련 저널

### way1) 크롤링

In [ ]:
# 1. 전기차 동향: cox automotive 분기별 insights
base_url = "https://www.coxautoinc.com/market-insights"

year = range(2023, 2025)
quarters = ['q1', 'q2', 'q3', 'q4']

urls1 = [f"{base_url}/{q}-{y}-ev-sales/" for y in year for q in quarters]

In [ ]:
urls1 = urls1[:(len(urls1)-1)]
urls1

['https://www.coxautoinc.com/market-insights/q1-2023-ev-sales/',
 'https://www.coxautoinc.com/market-insights/q2-2023-ev-sales/',
 'https://www.coxautoinc.com/market-insights/q3-2023-ev-sales/',
 'https://www.coxautoinc.com/market-insights/q4-2023-ev-sales/',
 'https://www.coxautoinc.com/market-insights/q1-2024-ev-sales/',
 'https://www.coxautoinc.com/market-insights/q2-2024-ev-sales/',
 'https://www.coxautoinc.com/market-insights/q3-2024-ev-sales/']

In [ ]:
data1 = []

for url in urls1:
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")

        # 기사 본문 추출 (예: 특정 클래스나 태그 선택)
        text = soup.find("div", {"class": "article-content"}).text.strip()

        # 발행 날짜 추출
        publish_date = soup.find("time").text.strip()  # 또는 적절한 태그/클래스

        # 데이터 저장
        data1.append({"URL": url, "Publish_Date": publish_date, "Content": text})

    except Exception as e:
        print(f"Failed to process {url}: {e}")

Failed to process https://www.coxautoinc.com/market-insights/q1-2023-ev-sales/: 'NoneType' object has no attribute 'text'
Failed to process https://www.coxautoinc.com/market-insights/q2-2023-ev-sales/: 'NoneType' object has no attribute 'text'
Failed to process https://www.coxautoinc.com/market-insights/q3-2023-ev-sales/: 'NoneType' object has no attribute 'text'
Failed to process https://www.coxautoinc.com/market-insights/q4-2023-ev-sales/: 'NoneType' object has no attribute 'text'
Failed to process https://www.coxautoinc.com/market-insights/q1-2024-ev-sales/: 'NoneType' object has no attribute 'text'
Failed to process https://www.coxautoinc.com/market-insights/q2-2024-ev-sales/: 'NoneType' object has no attribute 'text'
Failed to process https://www.coxautoinc.com/market-insights/q3-2024-ev-sales/: 'NoneType' object has no attribute 'text'


In [ ]:
# 2. 전기차 동향: EV Volumes
urls2 = ["https://ev-volumes.com/news/ev/byd-dominates-record-breaking-chinese-ev-market-in-september/", "https://ev-volumes.com/news/ev/is-europes-ev-decline-impacting-market-forecasts/"]

In [ ]:
data2 = []

for url in urls2:
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")

        # 기사 본문 추출 (예: 특정 클래스나 태그 선택)
        text = soup.find("div", {"class": "article-content"}).text.strip()

        # 발행 날짜 추출
        publish_date = soup.find("time").text.strip()  # 또는 적절한 태그/클래스

        # 데이터 저장
        data2.append({"URL": url, "Publish_Date": publish_date, "Content": text})

    except Exception as e:
        print(f"Failed to process {url}: {e}")

Failed to process https://ev-volumes.com/news/ev/byd-dominates-record-breaking-chinese-ev-market-in-september/: 'NoneType' object has no attribute 'text'
Failed to process https://ev-volumes.com/news/ev/is-europes-ev-decline-impacting-market-forecasts/: 'NoneType' object has no attribute 'text'


In [ ]:
# 4. 현대기아 뉴스: InsideEVs

In [ ]:
!pip install newspaper3k

In [ ]:
from newspaper import Article

In [ ]:
urls3 = ["https://insideevs.com/news/742182/hyundai-gm-ev-pickup/", "https://insideevs.com/news/741714/hyundai-ioniq-9-three-row/", "https://insideevs.com/news/739804/us-election-revolution-wait-see/",
         "https://insideevs.com/news/738597/hyundai-ev-drill-fire-lance/", "https://insideevs.com/news/737979/inster-cross-released-pricing-premium-hyundai/", "https://insideevs.com/news/733079/hyundai-inster-korea-reviews/",
         "https://insideevs.com/news/732778/2025-hyundai-5-n-free-home-charger/", "https://insideevs.com/news/732351/2025-ioniq-5-xrt/", "https://insideevs.com/news/731625/hyundai-n-vision-74-hydrogen-hybrid-sports-car/",
         "https://insideevs.com/news/731619/hyundai-hybrid-v2l/", "https://insideevs.com/news/731383/porsche-simulated-shifting-porsche-worse/", "https://insideevs.com/news/729924/vw-trinity-golf-ev-delay/", "https://insideevs.com/news/727588/hyundai-trump-lobbying-ev-subsidies/",
         "https://insideevs.com/news/726964/hyundai-ioniq-5-v2l-apartment/", "https://insideevs.com/news/725901/tesla-model-3-performance-review/", "https://insideevs.com/news/725962/hyundai-strong-ev-competition-2024/", "https://insideevs.com/news/725447/hyundai-inster-video-static/",
         "https://insideevs.com/news/725266/average-ev-price-less-tesla/", "https://insideevs.com/news/724328/hyundai-kia-ioniq-5-gameboy/", "https://insideevs.com/news/723963/critical-materials-2024-06-20/", "https://insideevs.com/news/707496/hyundai-casper-ev-europe/",
         "https://insideevs.com/news/692440/hyundai-ioniq-6-mechanic-review-video/", "https://insideevs.com/news/687728/hyundai-ioniq-kona-ev-free-level-2-charger-promotion/", "https://insideevs.com/news/679686/tesla-model-y-range-test-consumer-reports/", "https://insideevs.com/news/674304/hyundai-toyota-ev-ads-ban-uk/",
         "https://insideevs.com/news/673466/tesla-not-on-top-10-best-evs-list-2023/", "https://insideevs.com/news/673338/hyundai-motor-group-kia-genesis-electric-truck/", "https://insideevs.com/news/669391/hyundai-ioniq-5-ioniq-6-now-cheaper-lease-than-tesla-model-3/",
         "https://insideevs.com/news/666058/rivian-tesla-hyundai-seat-comparison-video/", "https://insideevs.com/news/664554/watch-2024-hyundai-ioniq-5-on-nurburgring/", "https://insideevs.com/news/658984/hyundai-finds-causes-for-ioniq-5-12v-battery-drain-reveals-fixes/",
         "https://insideevs.com/news/658919/hyundai-european-boss-surprised-at-success-of-ioniq5/", "https://insideevs.com/news/655865/hyundai-kona-n-electric-ioniq-5n-possibility/", "https://insideevs.com/news/652101/hyundai-usa-launches-evolve-ev-subscription-program-from-699-usd/"]

In [ ]:
data3 = []

for url in urls3:
  article = Article(url)
  article.download()
  article.parse()
  data3.append({"title": article.title, "article": article.text})

In [ ]:
df3 = pd.DataFrame(data3)

In [ ]:
date = ["20241125", "20241120", "20241105", "20241024", "20241018", "20240909", "20240906", "20240903", "20240828", "20240828", "20240826", "20240813", "20240723", "20240719", "20240709", "20240709", "20240703", "20240702", "20240701", "20240620", "20240206", "20231020", "20230921", "20230801", "20230628", "20230623", "20230622", "20230526", "20230508", "20230427", "20230324", "20230324", "20230306", "20230212"]

In [ ]:
df3

,title,article
0,GM And Hyundai EV Pickup Could Be In Development,General Motors and Hyundai are reportedly team...
1,Hyundai Ioniq 9: A Three-Row EV With 335 Miles...,"Meet the Hyundai Ioniq 9, the brand's first th..."
2,America's EV Revolution Enters 'Wait And See' ...,"It's Election Day in the U.S., which means mil..."
3,Hyundai’s Drill Lance Pokes A Hole Into An EV’...,Hyundai's logistics division came up with a de...
4,Hyundai Gave Its Smallest EV A Little More Off...,"The Hyundai Inster Cross adds skid plates, mat..."
5,"The Hyundai Inster Shows Tesla What A $25,000 ...",The Hyundai Inster gets a positive nod from Ko...
6,2025 Hyundai Ioniq 5 N Owners Can Get A Free H...,Hyundai is offering a complimentary home EV ch...
7,"2025 Hyundai Ioniq 5 Gets 'Off-Road' XRT Trim,...",The 2025 Hyundai Ioniq 5 will feature native T...
8,Hyundai N Vision 74: Radical Hydrogen 'Rolling...,"Hyundai's ""hydrogen hybrid"" sports car is conf..."
9,Hyundai Is Making Hybrids That Can Power Your ...,South Korean automaker Hyundai Motor Group is ...


In [ ]:
df3['date'] = date

In [ ]:
df3

,title,article,date
0,GM And Hyundai EV Pickup Could Be In Development,General Motors and Hyundai are reportedly team...,20241125
1,Hyundai Ioniq 9: A Three-Row EV With 335 Miles...,"Meet the Hyundai Ioniq 9, the brand's first th...",20241120
2,America's EV Revolution Enters 'Wait And See' ...,"It's Election Day in the U.S., which means mil...",20241105
3,Hyundai’s Drill Lance Pokes A Hole Into An EV’...,Hyundai's logistics division came up with a de...,20241024
4,Hyundai Gave Its Smallest EV A Little More Off...,"The Hyundai Inster Cross adds skid plates, mat...",20241018
5,"The Hyundai Inster Shows Tesla What A $25,000 ...",The Hyundai Inster gets a positive nod from Ko...,20240909
6,2025 Hyundai Ioniq 5 N Owners Can Get A Free H...,Hyundai is offering a complimentary home EV ch...,20240906
7,"2025 Hyundai Ioniq 5 Gets 'Off-Road' XRT Trim,...",The 2025 Hyundai Ioniq 5 will feature native T...,20240903
8,Hyundai N Vision 74: Radical Hydrogen 'Rolling...,"Hyundai's ""hydrogen hybrid"" sports car is conf...",20240828
9,Hyundai Is Making Hybrids That Can Power Your ...,South Korean automaker Hyundai Motor Group is ...,20240828


In [ ]:
urls4 = ["https://insideevs.com/news/742393/kia-phev-60-miles-electric-range/", "https://insideevs.com/news/741881/kia-trump-ev-tax-credit/", "https://insideevs.com/news/735373/kia-hack-cyber-security-researcher/",
         "https://insideevs.com/news/734188/kia-stinger-electric-successor-legacy/", "https://insideevs.com/news/734124/kia-pbv-electric-vans-europe/", "https://insideevs.com/news/732758/upcoming-popular-evs-google-search/",
         "https://insideevs.com/news/731619/hyundai-hybrid-v2l/", "https://insideevs.com/news/730672/south-korea-ev-fire-panic/", "https://insideevs.com/news/729924/vw-trinity-golf-ev-delay/",
         "https://insideevs.com/news/728862/kia-ev4-spotted-europe/", "https://insideevs.com/news/725266/average-ev-price-less-tesla/", "https://insideevs.com/news/724328/hyundai-kia-ioniq-5-gameboy/", "https://insideevs.com/news/720934/kia-ev3-why-it-matters/",
         "https://insideevs.com/news/720641/kia-ev3-why-fwd/", "https://insideevs.com/news/719554/2025-kia-ev6-facelift-bigger-battery/", "https://insideevs.com/news/683132/cheaper-kia-ray-ev-minicar-returns-korean-market-more-range-power/",
         "https://insideevs.com/news/681970/2025-kia-ev5-leaked-china-stays-true-concept/", "https://insideevs.com/news/680212/kia-stops-taking-ev9-reservations-canada/", "https://insideevs.com/news/673466/tesla-not-on-top-10-best-evs-list-2023/",
         "https://insideevs.com/news/673338/hyundai-motor-group-kia-genesis-electric-truck/", "https://insideevs.com/news/672899/kia-to-expand-electric-lineup-from-ev1-to-ev9/", "https://insideevs.com/news/672854/kia-ev-deliveries-have-already-begun-south-korea/",
         "https://insideevs.com/news/667551/kia-makes-2023-ev6-more-attractive-lease-purchase-us/", "https://insideevs.com/news/666499/kia-ev9-cold-weather-testing-battery-preconditioning/", "https://insideevs.com/news/659603/kia-ceo-confirms-ev9-gt-as-high-performance-variant-coming-in-2025/"]

In [ ]:
data4 = []

for url in urls4:
  article = Article(url)
  article.download()
  article.parse()
  data4.append({"title": article.title, "article": article.text})

In [ ]:
df4 = pd.DataFrame(data4)

In [ ]:
df4

,title,article
0,Kia Eyes PHEVs With 60 Miles Of Electric Range...,Kia's plug-in hybrids might offer 60 miles of ...
1,'It Would Just Be Dumb': Kia Exec On Plan To K...,Kia America's COO said that eliminating the ta...
2,Hackers Get Keys to Any Kia With Just A Licens...,A flaw in Kia's dealer system allowed for atta...
3,Kia Stinger Replacement Could Go Electric. Wil...,Kia is reconsidering resurrecting the Stinger ...
4,Kia’s Futuristic Electric Vans Have Arrived In...,Kia brings three different versions of its PV5...
5,Upcoming Electric Cars Dominate Google Search....,EVs top the list of the most searched-for upco...
6,Hyundai Is Making Hybrids That Can Power Your ...,South Korean automaker Hyundai Motor Group is ...
7,South Korea's EV Fire Panic: Explained,The world is watching as EV fires keep making ...
8,Volkswagen May Push Back Flagship Next-Gen 'Tr...,Car manufacturers have been a bit overly optim...
9,The Kia EV4 Aims To Undercut The Tesla Model 3,Kia wants to build a slew of models on the Hyu...


In [ ]:
date = ["20241127", "20241121", "20240927", "20240917", "20240917", "20240906", "20240828", "20240820", "20240813", "20240802", "20240702", "20240701", "20240524", "20240523", "20240514", "20230823", "20230816", "20230805", "20230623", "20230622", "20230620", "20230620", "20230517", "20230510", "20230329"]

In [ ]:
df4['date'] = date

In [ ]:
df4

,title,article,date
0,Kia Eyes PHEVs With 60 Miles Of Electric Range...,Kia's plug-in hybrids might offer 60 miles of ...,20241127
1,'It Would Just Be Dumb': Kia Exec On Plan To K...,Kia America's COO said that eliminating the ta...,20241121
2,Hackers Get Keys to Any Kia With Just A Licens...,A flaw in Kia's dealer system allowed for atta...,20240927
3,Kia Stinger Replacement Could Go Electric. Wil...,Kia is reconsidering resurrecting the Stinger ...,20240917
4,Kia’s Futuristic Electric Vans Have Arrived In...,Kia brings three different versions of its PV5...,20240917
5,Upcoming Electric Cars Dominate Google Search....,EVs top the list of the most searched-for upco...,20240906
6,Hyundai Is Making Hybrids That Can Power Your ...,South Korean automaker Hyundai Motor Group is ...,20240828
7,South Korea's EV Fire Panic: Explained,The world is watching as EV fires keep making ...,20240820
8,Volkswagen May Push Back Flagship Next-Gen 'Tr...,Car manufacturers have been a bit overly optim...,20240813
9,The Kia EV4 Aims To Undercut The Tesla Model 3,Kia wants to build a slew of models on the Hyu...,20240802


### way2) 수기

In [ ]:
# 1. 전기차 동향: cox automotive 분기별 insights
data1 = [
    {"date": "20230412",
     "title": "Another Record Broken: Q1 Electric Vehicle Sales Surpass 250,000, as EV Market Share in the U.S. Jumps to 7.2% of Total Sales",
     "article": """For more than a year now, the Industry Insights team at Cox Automotive has been expecting monthly new-vehicle sales to improve. In most of 2022, monthly sales volume was consistently stuck at around 1.1 million units as low new-vehicle inventory levels squeezed sales. Inventory has improved notably, and new-vehicle sales are increasing. Inventory volume in Q1 was up by 70% compared to the same timeframe last year. And what do we know about higher inventory? It often leads to higher sales.

Electric vehicle inventory—and choice—expanded notably in Q1 as well. A year ago, there were 34 electric vehicles to choose from. This year, the number jumped to 42, and choice is expected to expand further in every quarter of 2023. One note, the 42 count includes the GMC Hummer EV—2 were sold in March—and the Mazda MX-30, which is sold only in select markets and chalked up volume of 15 in Q1. Vinfast, a new player on the field, has also now been included.

Counting all EVs, including Hummers and MX-30s, the Kelley Blue Book team estimates EV sales in Q1 increased by 44.9% year over year and reached 258,882, a record quarter for the U.S. market. With Q1 in the books, the U.S. market is on course for annual EV sales to surpass 1 million for the first time in 2023, as predicted by Cox Automotive analysts this past December.

Tesla led the way in Q1, helped by sizeable price cuts. Tesla sales were up 24.6% year over year, and the EV pioneer continues to be both the industry’s top-selling luxury make and EV make by far. Tesla’s share of the EV market fell to 62.4%, down slightly from its full-year result in 2022—64.2%—and down significantly from the nearly 79% share Tesla commanded in 2020. Still, a 62% share of the EV market makes Tesla “the market.”

Chevy Bolt sales in Q1 were strong, placing the bowtie brand a distant No. 2 in EV sales; Ford was third. And interestingly, despite solid, popular products from both Hyundai and Kia, EV sales from the brands decreased last quarter. On the luxury side, Audi, BMW and Mercedes all posted strong year-over-year sales growth thanks to excellent new offerings.

New tax incentives pushed by the government are certainly spurring interest, and some confusion. But EVs continue to be the talk of the industry. Overall, the Cox Automotive team believes EV sales growth will outpace industry growth for the foreseeable future. EV share in Q1 hit 7.2% of new-vehicle sales, which is also a record. Many analysts talk about tipping points in the industry when volume quickly accelerates. Considering automaker investments and future product plans on the table, that tipping point may well be fast approaching, if not already here."""},
    {"date": "20230612",
     "title": "Electric Vehicle Sales in Q2 Strike Another Record, but Growth Ahead Will Be Hard Fought",
     "article": """Nearly 300,000 new electric vehicles (EVs) – full battery-electric vehicles – were sold in the U.S. in Q2, a record for any quarter and an increase of 48.4% from Q2 2022. Tesla, again, was the largest seller of EVs in the U.S., with more than 175,000 sold, an increase of 34.8% quarter over quarter. Sales growth at Tesla, and the EV segment overall, was aided by sizeable price cuts by some automakers (Tesla, really) and incentive levels well above the industry average. In June, the average price paid for an EV was down nearly 20% year over year. EV share of the U.S. market in Q2 was 7.2%, up from 5.7% a year ago and down from the high in Q1 of an upwardly revised 7.3%.

Tesla is the No. 1 seller of luxury vehicles in the U.S., but its share of EV sales continues to fizzle. In Q2, Tesla’s share fell below 60% for the first time, but the No. 2 seller of EVs in the U.S. – Chevrolet – is a distant second. Tesla outsold Chevrolet 10 to 1 in Q2. Ford and Hyundai are third and fourth behind Chevrolet. Newcomer Rivian had a good quarter, with more than 20,000 units sold. Tesla’s success, in many ways, is the Model Y, the leading EV in America. One of every three EVs sold last quarter was a Tesla Model Y. Add in the similar-sized Model 3, and those two products are half the electric vehicle business. The once-leading Model S is no longer the best-selling high-end EV. Sales last quarter were estimated at 5,257, down more than 40% year over year and well behind the newcomer: BMW i4 EV sedan, with 6,777 sales in Q2, the new boss when it comes to super-lux EV sedans.

EV sales growth and filling the gaps in infrastructure to support EV mobility continue to be the industry’s leading story. Cox Automotive expects the country to welcome 1 million new EVs to its roads in 2023, which will fuel the budding used EV market further. Sparked in part by government incentives, new battery production facilities are cropping up around the country, and factories are being retooled and rethought to support EV production. This fall, the United Auto Workers are negotiating their labor contracts with many automakers, and one topic is front and center: EVs. Even vehicle transportation and logistics – the trucking of EVs – is being redesigned in some part, as the current crop of EVs is heavier and harder to move than their ICE brethren.

And EV sales keep growing.

In a recent Cox Automotive survey, more than 50% of shoppers were interested in adding an EV to their stable. Being interested is easy, of course, but far fewer people actually buy. And in the same survey, 53% of consumers agreed that EVs will eventually replace traditional ICE-powered vehicles. Dealers were more cautious, with only 31% agreeing on an all-EV future. Dealers have a front-row seat to the many challenges ahead. And many dealers, recently, have been watching EV inventory building.

In late June 2023, the days’ supply of EVs was over 100, whereas industry-wide inventory levels were closer to 53 days. When it comes to EV sales, the market is likely heading into its Trough of Disillusionment (hat tip to the Gartner Hype Cycle), where collaboration across many parties will be necessary to push through. Building EVs is one thing, and many in the industry are proving excellent at that skill. Selling EVs is something different altogether. Yes, EV sales records will continue to be set, and EV growth will continue to outpace overall industry growth, but the days of 75% year-over-year growth are in the rearview mirror. The hard-growth days are ahead."""},
    {"date": "20231012",
     "title": "Another Quarter, Another Record: EV Sales in the U.S. Surpass 300,000 in Q3, as Tesla Share of EV Segment Tumbles to 50%",
     "article": """Electric vehicle (EV) sales volumes set another record in Q3, as total sales of battery-powered vehicles jumped past 300,000 for the first time in the U.S. market. Year-to-date EV sales through September reached just over 873,000, putting the market firmly on track to surpass 1 million for the first time ever. The milestone will likely be achieved in November.

Total EV sales in Q3, according to an estimate from Kelley Blue Book, hit 313,086, a 49.8% increase from the same period one year ago and an increase from the 298,039 sold in Q2. Most automakers posted sizeable gains over 2022, with Volvo, Nissan, Mercedes and Hyundai delivering increases above 200%, thanks mostly to new products entering the market. In Q3, 14 new EV models that did not exist one year ago were in the mix, including Chevrolet Blazer and Silverado EVs. (The new Chevy EVs had very low sales – just an initial few to mark a Q3 launch). The EV market is transforming, to be sure.

Electric vehicle sales accounted for 7.9% of total industry sales in Q3, a record and up from 6.1% a year ago and 7.2% in Q2. As Cox Automotive has been reporting, higher inventory levels, more product availability, and downward pricing pressure have helped spur continued linear growth of EV sales in the U.S. market. EV sales have now increased for 13 straight quarters.

EV transaction prices in Q3 were down significantly from 2022. In an attempt to increase sales volume, Tesla slashed prices, which are now down roughly 25% year over year. The price cuts have helped, as Tesla’s Q3 sales grew by 19.5% year over year, surpassing the industry’s overall growth rate of 16.3%. However, Tesla’s share of the EV segment continues to plunge, hitting 50% in Q3, the lowest level on record and down from 62% in Q1. The long-promised Cybertruck may reverse the downward trend, although competition from Ford, Rivian and Chevrolet will likely impact Tesla’s electric pickup volume potential.

The German luxury makes – Audi, BMW, and Mercedes – continue to rapidly increase sales of EV models. In Q3, BMW and Mercedes EV sales more than triple year-ago levels, while Audi posted an EV sales gain of 94%.

Among automakers selling internal combustion engine (ICE) and EV products, Audi, BMW and Volvo had the richest mix of EVs in Q3. BMW had the highest proportion, with over 15% of its sales being EVs, followed by Audi and Volvo, with over 12% of their sales being EVs, according to Kelley Blue Book estimates. Mercedes, Porsche and VW all had an EV mix in excess of 10% last quarter.

Q3 EV SHARE OF TOTAL BRAND SALES

In 2020, just three years back, EV sales in the U.S. passed 250,000 for the first time. This year, the market will jump past 1 million. In Q3, Tesla remains the undisputed leader in EV sales, with Ford a distant #2 on the list, selling just over 20,000 EVs.

Most analysts expect a flood of new EVs in the coming three years, with the number of available EV products likely to double by 2027. With this changing landscape, EV sales volume growth in the U.S. is expected to continue. Of late, product availability has grown exponentially, while consumer acceptance has grown in a more linear fashion. Those trends will likely continue, making for some very interesting market dynamics in the years ahead. Change is never easy."""},
    {"date": "20240109",
     "title": "A Record 1.2 Million EVs Were Sold in the U.S. in 2023, According to Estimates from Kelley Blue Book",
     "article": """A record 1.2 million U.S. vehicle buyers chose to go electric last year, according to estimates from Kelley Blue Book, a Cox Automotive company. More specifically, 1,189,051 new electric vehicles (EVs) were put into service as the slow shift to an electrified future continued unabated. In 2023, the EV share of the total U.S. vehicle market was 7.6%, according to Kelley Blue Book estimates. That is up from 5.9% in 2022.

EV sales in the fourth quarter set a record for both volume and share: 317,168 and 8.1%, respectively. And while records were set, the oft-reported slowdown is real. Q4 EV sales increased year over year by 40% – a strong result by any measure, except when compared to the growth the industry saw in previous quarters. The market posted a 49% gain in Q3, and EV sales were up 52% year over year in Q4 2022. By volume, EV sales in Q4 were higher than in Q3 by roughly 5,000 units. The EV market in the U.S. is still growing, but not growing as fast.

The Cox Automotive Economic and Industry Insights team is calling 2024 “the Year of More” when it comes to EVs. More new product, more incentives, more inventory, more leasing, more infrastructure – all the more will combine to push EV sales higher in the year ahead. The team forecasts EV share of the U.S. market in 2024 will reach 10%.

According to December transaction price data shared during the Cox Automotive Industry Insights Webcast early in January, transaction price parity between internal combustion engine vehicles and EVs looks more realistic in the coming years. Still, EVs remain expensive. Last month, the average price paid for a new EV was $50,789, according to Kelley Blue Book estimates. Shifts in tax incentives will both help and hurt buyers in the year ahead – fewer vehicles now qualify, but the ability for dealers to apply any tax credits at the point of sale will help. Generally speaking, shoppers are still hard-pressed to find a new EV with a manufacturer’s suggested retail price below $40,000. In fact, there were two last month: The Chevy Bolt and Nissan Leaf.

Tesla remains the undisputed champion of EV sales in the U.S., capturing 55% of the EV market in 2023. That’s down from 65% in 2022, but lower prices are clearly fortifying the company’s position. Tesla’s share increased in Q4 after falling to an all-time low in Q3. Notably, the Model Y accounted for 33% of all EVs sold in 2023. Tesla’s share of the total U.S. market reached an all-time high of 4.20% in calendar year 2023, a number that was not lost on Tesla boss Elon Musk and his humor. Tesla outsold VW, Subaru and others in 2023.

The German luxury brands have all lost share to Tesla over the past five years, but new products are certainly attracting luxury EV buyers back to the traditional Bavarian nameplates. Last year, 12.5% of BMW sales were EVs. Audi and Mercedes also increased their EV sales, which now account for 11% and 11.5% of total brand sales, respectively. Nearly all automakers should see their share of EV sales increase in the year ahead, and those not currently in the game will jump in.

Still, in the automobile business, nothing happens quickly. EV growth will continue to slow, and in the year ahead, we may even report the first quarter-over-quarter sales decline in more than three years. Regardless, the Cox Automotive Industry Insights team is forecasting more growth in the EV market. The momentum is there and is not going away."""},
    {"date": "20240411",
     "title": "EV Sales Growth Slows; Market Leader Tesla Stalls",
     "article": """Electric vehicle (EV) sales growth in the U.S. continues to slow, according to sales data analyzed by Kelley Blue Book. In the first quarter of 2024, Americans bought 268,909 new electric vehicles, according to Kelley Blue Book counts. EV share of total new-vehicle sales in Q1 was 7.3%, a decrease from Q4 2023.

While annual EV sales continue to grow in the U.S. market, the growth rate has slowed notably. Sales in Q1 rose 2.6% year over year, but fell 15.2% compared to Q4 2023. The increase last quarter was well below the previous two years.

In Q1 2023, EV sales volumes were up 46.4% year over year and 15.5% quarter over quarter. In Q1 2022, EV sales were higher by 81.2% year over year and 20.4% higher than the previous quarter.

“Electric vehicle sales in the U.S. declined during Q1 2024 – the first quarter-over-quarter downturn since Q2 2020,” said Stephanie Valdez Streaty, director of Industry Insights at Cox Automotive.

“As anticipated, Tesla’s sales took a hit, influencing the overall market dynamics. However, a few brands saw significant EV sales increases, achieving over 50% year-over-year growth. As noted in January, we are calling 2024, ‘the Year of More’. More new products, more incentives, more inventory, more leasing and more infrastructure will drive EV sales higher this year. Even so, we’ll continue to see ups and downs as the industry moves towards electrification.”

Analysts at Cox Automotive had expected a slowdown in EV sales growth. Segment growth typically slows as volume increases. This is certainly true with the market leader Tesla, which reported notably lower global deliveries in Q1 2024.

According to Kelley Blue Book estimates, Tesla sales in the U.S. were down 13.3% year over year – well below the typical double-digit growth that had become routine with the Tesla brand. Tesla’s share of the electric vehicle market in Q1 2024 was 51.3%, down from 61.7% one year earlier.

Though the overall year-over-year growth was minimal in Q1, nine manufacturers recorded more than 50% year-over-year growth in EV sales – BMW, Cadillac, Ford, Hyundai, Kia, Lexus, Mercedes, Rivian and Vinfast.

Notably, lower prices have supported EV sales volume in the U.S., particularly for key Tesla models. The average transaction price for a new EV in Q1 was $55,167, a 9.0% decrease compared to Q1 2023 and down 3.8% quarter over quarter. Tesla’s average transaction price was $52,315 in Q1, down roughly 13.5% year over year. However, lower prices did not generate higher volume.

Many automakers have followed Tesla’s lead and slashed prices. Incentive spending on EVs has increased notably in the past year, another sign of slowing demand. Leasing, too, has increased. In Q1, roughly 27% of all EVs were leased, more than double from the year before. With leasing, many buyers can qualify for the full $7,500 incentive the Inflation Reduction Act offers.

One bright spot in Q1: Strong EV sales from luxury makers, suggesting the EV market continues to be luxury-driven. Cadillac achieved a 499.2% year-over-year increase in electric vehicle sales due to robust sales of its Lyriq model. At Mercedes, EV sales were up 66.9%. BMW posted a 62.6% increase in EV sales compared to Q1 2023. At Audi, Q1 EV sales grew 28.8% year over year.

Meanwhile, sales of the most affordable EV in the U.S. – the Chevy Bolt – have been temporarily halted. Bolt sales fell 64.3% year over year in Q1, hitting just 7,040, as production stopped. A new version of the Bolt is expected to launch in 2025. On the non-luxury side, Ford achieved an 86.1% year-over-year increase in Q1 EV sales with the second-highest EV sales volume behind Tesla.

Cox Automotive forecasts EV sales in the U.S. to increase year over year in 2024, making this year the best year ever for EV sales. Analysts expect EV sales to reach roughly 10% of the market by the end of the year, up from 7.3% in the first quarter."""},
    {"date": "20240611",
     "title": "Despite Tesla Slide, EV Sales in Q2 Mark New Record",
     "article": """Electric vehicle sales in the U.S. grew by 11.3% year over year in the second quarter, reaching a record-high volume of 330,4631 units, according to new estimates from Kelley Blue Book. This growth was driven partly by improved availability, higher discounts and elevated levels of leasing. Total EV sales last quarter were higher than Q1 sales by 23% percent. In Q2, market leader Tesla sales volume declined by 6.3% year over year, but new products, notably from General Motors, helped lift overall volume higher. In the second quarter, Tesla’s share of EV sales fell below 50% for the first time in the U.S. to 49.7%.

The sales estimates from Kelley Blue Book show that electric vehicles accounted for approximately 8% of total new-vehicle sales in the second quarter, higher than the 7.1% share in the first quarter of the year and higher than the 7.2% recorded in Q2 last year.

Cox Automotive Industry Insights Director Stephanie Valdez Streaty commented: “EV sales exceeded expectations during a record-breaking quarter. Despite Tesla’s declining sales, with its EV sales share now below 50% for the first time, the overall competitive landscape for electric vehicles is intensifying. This increased competition is leading to continued price pressure, gradually boosting EV adoption. Automakers that deliver the right product at the right price and offer an excellent consumer experience will lead the way in EV adoption.”

While Tesla remains the dominant player in the EV segment, new electric offerings by mainstream competitors – and more aggressive pricing – continue to provide shoppers with interesting alternatives. Notable new entries in Q2 include the BMW i5, Cadillac Lyriq, Honda Prologue, and Kia EV9 SUV. Higher volumes of the Ford Mustang Mach-E and F-150 Lightning helped the Ford brand retain its No. 2 position, behind Tesla, in the EV market.

New EVs from General Motors are also helping grow industry volume. In Q2, GM added more than 21,000 EVs to the market. New models include electric versions of the Chevy Blazer, Equinox and Silverado, which tallied more than 10,000 sales in Q2, although the extra volume was not enough to offset the discontinued Chevy Bolt. In Q2, Chevrolet’s EV sales were lower by 19% year over year.  Other major players who lost ground year over year in Q2 include Mercedes-Benz, Polestar, Porsche, Volvo and VW.

Added Valdez Streaty, “We remain bullish on electric vehicle sales in the long term. The growth will, at times, be very slow, as all-time horizons in the automobile business are vast, but the long-term trajectory suggests that higher volumes of EVs will continue over time. As EV infrastructure and technology improve, and more models are launched, many shoppers sitting on the fence will eventually choose an EV.”"""},
    {"date": "20241011",
     "title": "Electric Vehicle Sales Mark Another Record in Q3, Thanks to Higher Incentives, More Choices",
     "article": """According to new estimates from Kelley Blue Book, electric vehicle (EV) sales in the U.S. grew by 11% year over year in the third quarter and reached record highs for both volume and market share. According to the latest counts, an estimated 346,3091 EVs were sold in Q3 2024, a 5% increase from Q2. The EV share of sales in Q3 hit 8.9%, the highest level recorded and an increase from 7.8% in Q3 2023.

“While year-over-year growth has slowed, EV sales in the U.S. continue to march higher,” said Stephanie Valdez Streaty, director of Industry Insights at Cox Automotive. “The growth is being fueled in part by Incentives and discounts; but as more affordable EVs enter the market and infrastructure improves, we can expect even greater adoption in the coming years.”

Electric vehicle incentives and discounts were elevated in Q3, which helped fuel higher sales. In Q3, incentives averaged more than 12% of the average transaction price (ATP), much higher than the industry-wide average of approximately 7%.

The “leasing loophole” was also generously applied, as leasing allows all EV buyers to qualify for generous government-supported incentives. Electric vehicle prices were higher year over year in Q3 but by less than 1%. The average price paid for an EV in Q3 was just over $57,000, a premium of approximately 19% compared to the industry-wide ATP of just over $48,000.

After seeing sales fall year over year in the first half, market leader Tesla returned to growth mode in Q3, with sales up 6.6%, thanks mostly to the newly introduced Cybertruck. Despite a six-figure price tag – average transaction price in September was north of $116,000, according to Kelley Blue Book estimates – more than 16,000 Cybertrucks were sold in Q3. In fact, Cybertruck outsold every other available EV except for two – Tesla’s popular Model Y and Model 3.

General Motors EV sales jumped significantly in Q3, up nearly 60% to 32,095. While still a distant second to market leader Tesla, GM improved its standing in the EV race thanks to strong sales from three of its four core brands – Cadillac, Chevrolet and GMC. Hyundai Motor Group fell behind GM and into third place, with sales generally flat year over year at 29,609. Ford was fourth in sales with 23,509, and Honda and Acura contributed more than 15,000 EV sales in the quarter, thanks to two new models. A year ago, American Honda EV sales were zero.

Cox Automotive was expecting 2024 to be a “year of more” for EVs in the U.S. – more products, more sales, more discounting, and more curious buyers jumping into the fray. With improving infrastructure, far more choices, and excellent deals to be had, the team expects further growth in the months ahead – a 10% share is well within reach."""}]


In [ ]:
df1 = pd.DataFrame(data1)

In [ ]:
df1

,date,title,article
0,20230412,Another Record Broken: Q1 Electric Vehicle Sal...,"For more than a year now, the Industry Insight..."
1,20230612,Electric Vehicle Sales in Q2 Strike Another Re...,"Nearly 300,000 new electric vehicles (EVs) – f..."
2,20231012,"Another Quarter, Another Record: EV Sales in t...",Electric vehicle (EV) sales volumes set anothe...
3,20240109,A Record 1.2 Million EVs Were Sold in the U.S....,A record 1.2 million U.S. vehicle buyers chose...
4,20240411,EV Sales Growth Slows; Market Leader Tesla Stalls,Electric vehicle (EV) sales growth in the U.S....
5,20240611,"Despite Tesla Slide, EV Sales in Q2 Mark New R...",Electric vehicle sales in the U.S. grew by 11....
6,20241011,Electric Vehicle Sales Mark Another Record in ...,According to new estimates from Kelley Blue Bo...


In [ ]:
# 2. EV Volumes
data2 = [
    {"date": "20241118",
     "title": "On song BYD dominates record-breaking Chinese EV market in September",
     "article": """Chinese electric vehicle (EV) deliveries soared in September, as BYD continued its domestic market domination. José Pontes, data director at EV Volumes, looks into the figures with Autovista24 special content editor Phil Curry.

Registrations of EVs, including battery-electric vehicles (BEVs) and plug-in hybrids (PHEVs) set a new record in China in September. More than 1.1 million units were delivered in the month, a 51% increase year on year. This is even more spectacular when considering the overall market improvement in the month was just 5%.

Growth came from both powertrain types. BEVs were up 29% in September, to around 644,000 units, while PHEVs jumped 97% to around 479,000 units. This was the fourth month in a row that PHEVs broke their volume record.

September’s strong results saw EVs take a 53% share of the overall Chinese new-car market. BEVs alone accounted for 31% of all deliveries. In terms of EV share, BEVs held 57% of registrations in the month, while PHEVs had 32%. The remaining 11%  share was captured by extended-range models (EREVs).

In the first nine months of 2024, around 7.3 million EVs have been delivered to customers, a significant rise over the 5.4 million units that were registered in the same period during 2023.

Looking at the market share between January and September, EVs held 47%, with 27% of this belonging to BEVs. Compared with 12 months prior, the 2023 plug-in share was 37%, with BEVs making up 25% overall. This means that while BEVs have only improved market share by two percentage points (pp), PHEVs are growing faster. The technology’s share increased by around 8pp year on year.

With this growth, the year should end with EVs holding around 50% of the market. This means the push for full electrification by around 2030 is still heading in the right direction."""},
    {"date": "20241002",
     "title": "Is Europe’s EV decline impacting market forecasts?",
     "article": """With Europe’s light vehicle market slowing, how have electric vehicle (EV) forecasts been affected? Neil King, head of forecasting at EV Volumes, presents the latest outlook with Autovista24 special content editor Phil Curry.

Europe’s light-vehicle market, made up of passenger cars and light-commercial vehicles (LCVs), has experienced a rollercoaster ride this year. Major markets across the continent enjoyed relatively consistent growth following the end of the supply-chain crisis in August 2022.

However, this year has seen instability, with strong overall increases offset by large declines. With high interest rates and the cost-of-living crisis, the automotive market is in a state of flux.

EV Volumes has reduced its forecast for Europe’s light-vehicle market in 2024. It now expects growth of 2.4% across the continent by the end of the year, down from the 2.6% forecast in June 2024.

At almost 15.1 million units, this falls far short of the 18 million light vehicles registered in Europe in 2019. Moreover, EV Volumes does not see the European market returning to this level during the current forecast horizon, up to 2035."""},
    {"date": "20240919",
     "title": "Which EV manufacturers performed the best in 2024 so far?",
     "article":""""Which electric vehicle (EV) manufacturers stood out in the first half of 2024? EV Volumes founder, Roland Irle, reviews the most successful OEMs in the electric light-vehicle market with Autovista24 editor Tom Geggus.

Covering both passenger cars and light-commercial vehicles (LCVs), the global light-vehicle market saw mixed performances from EV manufacturers in the first half of 2024. One of the most noticeable trends was that OEM success followed regional growth patterns.

Many Chinese vehicle makers recorded impressive growth between January and June. Compared to the restrained results of most Western OEMs, the likes of Geely, Changan, Li Auto, Seres (Aito) and Chery did particularly well.

BYD’s year-on-year volume growth slowed to 26%, compared to the 62% surge it recorded last year. Despite this, the manufacturer remained the global EV-market leader.

Its product portfolio featured 37 nameplates for sale in the first half of 2024. This included battery-electric vehicles (BEVs), plug-in hybrids (PHEVs) and extended-range electric vehicles (EREVs). However, BYD is not yet present across all vehicle segments and it does not offer mini-EVs. It is also late to the EREV party.

BYD is also China’s largest OEM by production and sales volumes, despite only offering BEV and PHEV models. It sold roughly 161,000 vehicles outside China, and only some 17,000 in Western and Central Europe. South America, ASEAN, the Middle East and Russia were BYD’s largest export regions, with combined sales of roughly 114,000 units.

Hyundai Motor Company saw its sales stay flat in the first half. However, Kia was able to make slight gains over Hyundai. Launched in the summer of 2023, the Kia EV9 recorded some 19,000 sales in the first six months of 2024.

Otherwise, model volume changes were small. The only exceptions were the Kia Bongo and Hyundai Porter, which saw the end of generous electric LCV grants in South Korea. This resulted in a loss of some 17,500 sales. Dropping deliveries in Europe and South Korea were balanced by good growth in the US. In China, the OEM sold only around 3,000 units of the Kia EV5.

Li Auto, known for its large range-extender-equipped SUVs, saw continued sales growth in the first half. Following the launch of the L6 and increasing sales in Russia, the manufacturer managed to record 14,400 unit sales."""},
     {"date": "20240905",
     "title": "Is the global EV market slowing down?",
     "article": """New electric vehicle (EV) sales across the world continued to grow across the first half of 2024. But did the market show signs of slowing? EV Volumes founder, Roland Irle, explores the progress of plug-ins with Autovista24 editor Tom Geggus.

The global new light-vehicle market, including passenger cars and light-commercial vehicles, grew by 3.7% in the first half of 2024. Within this, combined battery-electric vehicle (BEV) and plug-in hybrid (PHEV) sales increased by 22% year on year to 1.35 million units.

As confirmed by EV Volumes data, deliveries of new EVs did grow relatively consistently between January and June 2024. Because of the Chinese New Year, February was the only month not to record double-digit growth in the first half.

This consistency and competitively high level of growth appears promising at first. However, it does represent a slowdown from the same point last year. In the first half of 2023, the global EV market saw deliveries climb by 35% year on year."""},
     {"date": "20240819",
     "title": "How have global EV forecasts reacted to strong headwinds?",
     "article": """Global electric vehicle (EV) sales are experiencing turbulence, but is this enough to affect market trajectories? Neil King, head of forecasting at EV Volumes (part of J.D. Power), presents the latest outlook with Autovista24 special content editor Phil Curry.

Covering both passenger cars and light-commercial vehicles, the latest light-vehicle forecast from EV Volumes expects a 0.5% rise in total sales this year when compared with 2023.

This follows a weaker outlook in the non-Triad region, as well as downgraded expectations for Europe and North America. However, China did help compensate for this with an improved outlook.

Focusing on powertrain performance, the EV market has faced headwinds this year. Public opinion has suffered with talks of registration slowdowns, the cancelling of incentives and the implementation of provisional import tariffs.

Combined battery-electric vehicle (BEV) and plug-in hybrid (PHEV) volumes grew by 35% globally in 2023, reaching 14.2 million units. This equated to a 16.7% market share, up from 13.6% in 2022."""},
     {"date": "20240725",
     "title": "How is Europe’s EV slowdown affecting its market forecast?",
     "article": """Europe’s electric vehicle (EV) market has slowed in the first half of 2024. But how will it affect the market this year and beyond? Neil King, head of forecasting at EV Volumes, presents the latest forecast with Autovista24 special content editor Phil Curry.

Europe’s new light-vehicle market, made up of passenger cars and light-commercial vehicles (LCVs) is undergoing a difficult year in 2024, following its bounce back from various challenges last year.

The passenger-car market has seen slow growth, with declines in March and May, while the LCV market is currently performing well, but offers smaller volumes into the new light-vehicle sector.

The latest figures are compared to a period, however, where volumes increased rapidly, following the impact of a supply-chain crisis in 2021 and 2022. This was exacerbated by the war in Ukraine, and the fallout from the COVID-19 pandemic.

Yet, the market this year is more subdued, due to high interest rates and the continuing cost-of-living crisis.

With this in mind, EV Volumes has revised its forecasts from March 2024. The European new light-vehicle market is now expected to grow by 2.6% this year, a lower figure than the 3.1% improvement forecast earlier this year."""},
     {"date": "20240718",
     "title": "Europe’s medium and heavy-duty truck markets continue to electrify",
     "article": """The electrification of Europe’s medium and heavy-duty truck sector is growing, with more models available, enabling increased registrations and lower emissions. Przemek Kolasa, data manager for commercial vehicles at EV Volumes, explores the figures with Autovista24 special content editor Phil Curry.

The EU transport sector is responsible for more than a quarter of greenhouse gas emissions in the bloc. However, in the commercial-vehicle segment, the transition to zero-emission mobility is well underway.

Domestic vehicle manufacturers are contributing to this move, working heavily on the electrification of their products. Most have already launched zero-emission vehicles (ZEVs) for different commercial applications.

However, there is a greater need to accelerate this transition. In February 2024, the European Commission increased the target reductions in CO2 emissions for heavy-duty commercial vehicles. In 2030, levels must be 45% lower than those recorded in 2019, while this increases to a 65% reduction in 2035, and a 90% reduction in 2040.

Helping the market is the development of national strategy frameworks, which implement EU regulations on the expansion of the bloc’s charging and hydrogen infrastructure.

This is aimed at enabling the rollout of fast chargers, reducing the charging times for carriers. The medium-duty sector will benefit from this the most, with EV Volumes data showing that the average range of a vehicle in this segment is 220km.

For heavy-duty trucks, the average range is 330km on a single charge. To help fleets in this market, a transition from the current Combined Charge System to the Megawatt Charge System (MCS) is being studied. This will help to drastically reduce charging times for larger vehicles, as the MCS can deliver up to 3.5 megawatts of power."""},
     {"date": "20240613",
     "title": "How popular are electric buses in Europe?",
     "article": """Europe’s electric-bus market ended 2023 on a high, but has this growth continued into the first quarter of 2024? Przemek Kolasa, data manager for commercial vehicles at EV Volumes, examines the trends with Autovista24 special content editor Phil Curry.

Made up of full-electric and plug-in hybrid vehicles, Europe’s electric-bus market is growing steadily. Between 2021 and 2023, these models increased their hold on the new-bus market by seven percentage points, capturing 18% at the end of last year.

Full hybrid, alternative fuels (including liquid-petroleum gas, compressed-natural gas and ethanol) and hydrogen fuel-cell (FCEV), held respective shares of 11%, 10% and 0.5%. Between 2021 and 2023, they increased their hold by 2pp, 1pp and 0.07pp respectively.

Diesel buses still dominated the market, holding a 60% share at the end of 2023. However, this share has dropped 11pp from 2021."""},
     {"date": "20240527",
     "title": "Global EV growth forecast in 2024, but challenges remain",
     "article": """Despite current market conditions leading to amendments in global electric vehicle (EV) forecasts, the market is still expected to improve. Neil King, head of forecasting at EV Volumes (part of J.D. Power), presents the outlook with Autovista24 special content editor Phil Curry.

The global light-vehicle market, made up of passenger cars and light-commercial vehicles (LCVs) grew by 10% year on year in 2023. However, the numbers were still below those seen in 2019. This highlights the ongoing impact of the COVID-19 pandemic, supply shortages and the cost-of-living crisis.

While there was double-digit growth in Europe and North America (combining the US and Canada), gains were more subdued in China and the non-Triad region. Retail sales in China recovered from a sharp decline in 2022. The country’s government is seeking to stimulate an economy suffering from a struggling property sector, a lacklustre stock market, and high youth unemployment."""},
     {"date": "20240422",
     "title": "Tesla dominates European EV market in February",
     "article": """Electric vehicle (EV) registrations in Europe recorded double-digit growth in February, as Tesla dominated the market. José Pontes, data director at EV Volumes, evaluates the figures with Tom Hooker, Autovista24 journalist.

A total of 202,542 EVs took to the road in Europe during February, recording a growth of 10% year on year. This meant battery-electric vehicles (BEVs) and plug-in hybrids (PHEVs) accounted for 20% of deliveries in the overall market in February and the year-to-date.

BEVs achieved a 13% share of all passenger-car registrations during the month. The powertrain saw deliveries rise 10% year on year, despite a lack of mass-market EVs. Models that may help boost the sector, such as the Renault 5 and Citroen e-C3, are not yet available in Europe.

Additionally, many markets are still suffering from the ending of incentives. The negative incentive impact is expected to ease in the second quarter of 2024. However, significant BEV growth is not predicted until the second half of the year.

Elsewhere, PHEVs registrations increased 9% year on year. Meanwhile, hybrids (including full and mild powertrains) enjoyed a surge of 24%. The technology accounted for 29% of all new-car registrations in February, meaning almost half of the European market is now electrified.

EV and hybrid growth has come at the expense of diesel, which endured a delivery drop of 5% compared to one year ago, holding 12% of the market. This contrasts with its 50% share in 2015."""},
     {"date": "20240327",
     "title": "Growth forecast for European EV market despite incentive impact",
     "article": """Even with incentive changes, Europe’s new electric vehicle (EV) market is still forecast to grow. Neil King, head of forecasting at EV Volumes (part of J.D. Power), presents the outlook.

Europe’s new light-vehicle market, made up of passenger cars and light-commercial vehicles (LCVs), grew by 13.9% year-on-year in 2023. The sector was aided by improving supply, which reduced order backlogs created by component shortages in previous years. However, order intake is currently subdued because of high interest rates and cost-of-living increases.

EV Volumes forecasts that Europe’s combined new passenger-car and LCV market will grow by 3.1% this year. This equates to 15.2 million units, far short of the 18 million light vehicles registered in 2019. The European market is unlikely to return to this level anytime soon and is not expected to surpass this figure before 2035."""},
     {"date": "20240219",
     "title": "What was the most popular EV worldwide in 2023?",
     "article": """The global electric vehicle (EV) market broke records throughout 2023. Leading this charge was the Tesla Model Y. José Pontes, data director at EV-volumes.com, unpacks the year and its most popular performers.

New EVs, consisting of battery-electric vehicles (BEVs) and plug-in hybrids (PHEVs), saw global registrations jump 35% year on year in 2023. This allowed the electric market to end above the 13 million mark for the first time.

Plug-in models made up a record 22% of the entire new-car market in December, with BEVs accounting for 15% alone. This pushed 2023’s total EV share to 16%, a small rise from 14% in 2022. BEVs made up 11% of registrations worldwide last year, up from 10% in the previous year. It is worth considering that the overall global new-car market experienced double-digit growth in 2023.

PHEVs (up 47%) saw registrations grow more quickly across 2023 than BEVs (up 30%). This meant the hybrid powertrain increased its share of the EV market, reaching 31%, up from 28% in 2022. The PHEV share has been fluctuating between 26% and 31% since 2018, supporting the notion that the technology could remain relevant for a while."""},
     {"date": "20240122",
     "title": "Global EV Sales for 2023",
     "article": """Global EV sales continued as expected by us at the beginning of 2023. A total of 14,2 million new Battery Electric Vehicles (BEV) and Plug-in Hybrids (PHEV) were delivered during 2023, an increase of +35 %. 10 million were pure electric BEVs and 4,2 million were Plug-in Hybrids (PHEV) and Range Extender EVs (EREV).

Media reporting that EV adoption is in reverse is not evident in our monthly tracking. Global deliveries of BEVs and PHEVs increased by 3,66 million units over 2022 and all monthly sales were between 8 % and 52 % higher than in the year previous. 5 months of 2023 witnessed new all-time highs for EV sales, Market share increases were consistent throughout the year. The exception was Germany, Europe’s largest EV market, where a series of significant cuts in EV grants lead to s decline in EV sales and market shares.  European EV registration were up by a meager +17 % vs 2022; excluding Germany, the increase was +32 %. China EV sales increased by +36 % in 2023 y/y, EV sales in USA and Canada were +46 % higher than the year previous.  EV sales outside the aforementioned markets increased by +81 %, albeit from a low base.

In most countries the EV increases were accompanied by a strong recovery in overall light vehicle markets, increasing by +10,5 % globally. EV adoption does not follow auto market slumps and recoveries in lockstep. Successful EV launches, financial incentives, improving charging infrastructure and environmentalism are stronger forces. Slowing overall auto sales typically lead to faster EV share increases while strong auto market recoveries slow down EV share gains.

EV shares still continued to increase in most markets. BEVs (11,1 %) and PHEVs (4,7 %) stood for 15,8 % of global light vehicle sales in 2023, compared to 13,0 % in 2022. Norway had the highest market share of EVs in 2023 (BEVs 72 % + PHEVs 7 %), followed by Hong Kong and the Nordic countries in Europe. China had 33,9 %, Europe 21,4 % and USA 9,4 %. The fastest growing markets were Turkey with 86 600 units, up +805 %, Brazil with 50 400 units, up +359 %, Thailand with 89 000 units, up +328 %, Malaysia with 10 400 BEV & PHEV units, +886 % and Australia with 93 00 units, up +141 %.

PHEVs stood for 29,6 % in the global BEV-PHEV mix in 2023 compared to 27,2 % in 2022, a deviation from the long-term trend. The PHEV volume increase was 1,33 million units, most of it (1,2m) in China. High sales of BYD PHEVs and the renaissance of Range Extender Electric Vehicles (EREVs) are the main reason. EREVs have larger batteries and longer all-electric range than PHEVs, carrying a small petrol engine as a “back-up” generator. The Chevrolet Volt and the BMW i3 were early examples of this concept, the current EREV leader is Li Motor with 376 000 units delivered in 2023. Total EREV sales were 705 900 units, of which 98 % in were sold in China. Sales of Fuel Cell Vehicles (FCEV) in the light vehicle sector have declined by -38 % vs 2022 and stayed below 10 000 units annually. Current sales are from five vehicle models and most sales are in South Korea and USA. We estimate their current population to be about 60000 units.

BYD increased sales to 3 million units (+62 % y/y), remaining as the #1 in the global EV sales ranking, with their 1,44 million PHEV sales included. Counting BEVs only, Tesla still leads with 1,81 million units delivered in 2023, an increase of 38 % over 2022. A common observation is that pure EV players grew their sales faster than legacy OEMs.

By the end of 2023 we expect 40 million EVs in operation, counting light vehicles, 70 % of which are BEVs and 30 % PHEVs. For 2024, we expect sales of 17,8 million EVs, growth of +25 % over 2023, with BEVs reaching 12,8 million units and PHEVs 5 million units."""},
     {"date": "20241214",
     "title": "The green transition of electric trucks and buses",
     "article": """As electric vehicles (EVs) begin to make serious inroads into the passenger-car market, a green transition is also underway for heavy-duty trucks and buses. Przemek Kolasa, data manager for commercial vehicles at EV-volumes.com, examines the progress.

As a large contributor to global greenhouse gas levels, the commercial-vehicle sector is looking towards zero-emission possibilities. The development of battery-electric trucks and buses will lead to significantly cleaner mobility within logistics and passenger transport."""},
     {"date": "20231121",
     "title": "EVs Forecast to Account for Two Thirds of Global Light-Vehicle Sales in 2035",
     "article": """We forecast that global EV sales, comprising battery-electric vehicles (BEVs) and plug-in hybrids (PHEVs), will reach 14.1 million units in 2023. This equates to EV sales growth of 34% compared with 2022.

Meanwhile, the entire light-vehicle market, which includes passenger cars and light-commercial vehicles, is only predicted to rise by 9%. EVs are therefore expected to take a higher 16% share of the global light-vehicle market this year.

This market penetration rate is slightly lower than predictions made earlier in the year, especially as the 2023 EV share in China is now forecast to be 33%, down from 35% previously. However, over 300,000 additional EV sales are now expected globally in 2023, largely thanks to a stronger outlook for total light-vehicle sales in China.

EV uptake has weakened in North America, while recent subsidy cuts in European markets like Germany, Sweden and the UK have damaged demand, particularly for PHEVs. Accordingly, the global EV share is predicted to be lower throughout the decade than previously forecast, reaching 23.5% in 2025, then 45.3% in 2030, and 68.4% in 2035.

The unit volume of global EV sales is set to triple from 10.5 million in 2022 to over 31 million in 2027. It is expected to more than double to over 74.5 million units in 2035.

The number of EVs in operation will rise rapidly. But with 1.33 billion light vehicles currently on the road, EV-volumes forecasts that by the end of 2030, electric vehicles will only account for 15% of the global parc, assuming normal scrappage rates.

This will increase to nearly 30% by the end of 2035, with EVs only accounting for half of all operational light vehicles by 2042."""},
     {"date": "20230817",
     "title": "Global EV Sales for 2023 H1",
     "article": """Global EV sales continue strong. A total of 6 million new Battery Electric Vehicles (BEV) and Plug-in Hybrids (PHEV) were delivered during the first half of 2023, an increase of +40 %. 4,27 million were pure electric BEVs and 1,76 million were PHEVs. Preliminary July results show +40 % growth again. The regional growth pattern has shifted: China EV sales increased by +37 % in 2023 H1 y/y, compared to +82 % in 2022 vs 2021. Sales in Western and Central Europe were up +28 % in H1 compared to just +15 % growth in 2022. EV sales in USA and Canada are +50 % higher YTD to June than last year.  EV sales outside the aforementioned markets increased by 102 %, albeit from a low base.  Overall vehicle markets saw a considerable recovery, with +17 % y/y growth in Europe in H1 but weaker and more volatile in China. The global light vehicle market was 11 % higher in 2023 H1 than in 2022 H1 but still trailed the 2015-2019 average by five million units on an annualized basis.EV shares continued to climb in all markets. BEVs (10 %) and PHEVs (4,1 %) stood for 14,1 % of global light vehicle sales at the close of H1, compared to 11,3 % in 2022 H1. Norway had the highest market share of EVs in H1 (BEVs 75 % + PHEVs 6 %), China had 30,5 %, Europe 19,7 % and USA 8,7 %. The fastest growing markets were Indonesia with 6000 BEV & PHEV units for H1, +886 %, Thailand with 39 500 units, up +370 %, Turkey with 12 300 units, up +326 % and Australia with 47 400 units, up +248 %.

PHEVs stood for 29,6 % in the BEV-PHEV mix in 2023 H1 compared to 27,2 % in 2022. The PHEV volume increase was 891 000 units in H1, most of it (754k) in China. High sales of BYD PHEVs and the renaissance of Range Extender Electric Vehicles (EREVs) are the main reason.  Also, mini-EVs like the Wuling Hongguang found fewer buyers in a tougher economic environment. As they are all BEVs their decline is dragging down the BEV share within China’s New Energy Vehicle (NEV) volume.

BYD nearly doubled H1 sales to 1,25 millionb units, making it #1 in the global EV sales ranking, with their 633 000 PHEV sales included. Counting BEVs only, Tesla still leads by a wide margin with 889 000 units delivered in H1.For the full year of 2023, we expect sales of 14 million EVs, growth of +33 % over 2022, with BEVs reaching 10 million units and PHEVs 4 million units. By the end of 2023 we expect 40 million EVs in operation, counting light vehicles, 70 % of which are BEVs and 30 % PHEVs. Sales of Fuel Cell Vehicles (FCEV) in the light vehicle sector have declined by -25 % so far this year and remain below 20 000 units annually. Current sales are from five vehicle models and most sales are in South Korea and USA. We estimate their current population to be about 60 000 units."""},
     {"date": "20230206",
     "title": "Global EV Sales for 2022",
     "article": """Global EV sales continued strong. A total of 10,5 million new BEVs and PHEVs were delivered during 2022, an increase of +55 % compared to 2021. The regional growth pattern is shifting, though. Following 2 years of steep sales increases in Europe, EVs gained only +15 % over 2021 there. Weak overall vehicle markets and persistent component shortages have taken their toll, exacerbated by the war in Ukraine. EV sales in USA and Canada increased by 48 % year-on-year, despite a weak overall light vehicle market which plunged by 8 % during 2022 y/y. The 2nd half of 2022 saw a cautious recovery of auto markets as numbers compared to the low results of 2021 H1. Global light vehicle sales for 2022, 81 million units, were still -0,5 % lower than in 2021 and -15 % below pre-2020 levels.

China NEV sales defied all headwinds the country faced otherwise (real estate crisis, Covid outbreaks and lock-downs) and increased by another +82 % year-on-year. BYD more than tripled sales to 1,85 million units, making it the #1 in the global sales ranking, if their 944 500 PHEV sales are included. Counting BEVs only, Tesla still leads by a wide margin with 1,31 million units delivered in 2022.

PHEVs stood for 27 % of global Plug-in sales in 2022 compared to 29 % in 2021. While their sales volumes still increased, their share in the PEV mix is in decline, facing headwinds from incentive cuts and improving BEV offers. Sales growth is increasingly depending on the degree of electrification. While BEVs grew by +59 % and PHEVs by +46 %, non-chargeable Full Hybrids grew by +15 % and Mild Hybrids by +1 % y-o-y. Global sales of vehicles which can be charged from the grid (10,5 m) were higher than for non-chargeable vehicles (8,4 m) for the first time in 2022. ICE-only vehicle sales declined by -7 %; their share in global light vehicle sales is 76,8 %, compared to 82,2 % in 2021. FCEV remain irrelevant for the electrification of light vehicles; their deliveries stagnated at 15 400 units in 2022, which is 0,02 % of the global, annual light vehicle volume.

Rapid EV adoption in weak auto markets has boosted EV shares further. BEVs (9,5 %) and PHEVs (3,5 %) stood for 13 % of global light vehicle sales in 2022, compared to 8,3 % in 2021. Norway had the highest market share of EVs in H1 (BEV 71 % + PHEV 8 %), China had 27 %, Europe 20,8 % and USA 7,2 %. The fastest growing markets were Indonesia (from 1k to 10k), India with +223 % to 50k, nearly all BEVs, New Zealand +151 % to 23k for 20 % market share. EV supply and adoption is now spreading rapidly into the global south. November and December of 2022 saw demand distortions by coming reductions of EV grants in Europe and China and the IRA in the US for 2023. Demand for EVs and ICEs alike were pulled into 2022 or pushed into 2023. We expect irregular EV sales and shares in several countries for Q1 of 2023. For the full year of 2023, we expect sales of 14,3 million EVs, a growth of 36 % over 2022, with BEVs reaching 8 million units and PHEVs 2,6 million units. By the end of 2023 we expect nearly 40 million EVs in operation, counting light vehicles, 73 % are BEVs and 27 % PHEVs."""}

]

In [ ]:
df2 = pd.DataFrame(data2)

In [ ]:
df2

,date,title,article
0,20241118,On song BYD dominates record-breaking Chinese ...,Chinese electric vehicle (EV) deliveries soare...
1,20241002,Is Europe’s EV decline impacting market foreca...,"With Europe’s light vehicle market slowing, ho..."
2,20240919,Which EV manufacturers performed the best in 2...,"""Which electric vehicle (EV) manufacturers sto..."
3,20240905,Is the global EV market slowing down?,New electric vehicle (EV) sales across the wor...
4,20240819,How have global EV forecasts reacted to strong...,Global electric vehicle (EV) sales are experie...
5,20240725,How is Europe’s EV slowdown affecting its mark...,Europe’s electric vehicle (EV) market has slow...
6,20240718,Europe’s medium and heavy-duty truck markets c...,The electrification of Europe’s medium and hea...
7,20240613,How popular are electric buses in Europe?,Europe’s electric-bus market ended 2023 on a h...
8,20240527,"Global EV growth forecast in 2024, but challen...",Despite current market conditions leading to a...
9,20240422,Tesla dominates European EV market in February,Electric vehicle (EV) registrations in Europe ...


In [ ]:
# urls2 = ["https://ev-volumes.com/news/ev/byd-dominates-record-breaking-chinese-ev-market-in-september/", "https://ev-volumes.com/news/ev/is-europes-ev-decline-impacting-market-forecasts/",
#          "https://ev-volumes.com/news/ev/which-ev-manufacturers-performed-the-best-in-2024-so-far/", "https://ev-volumes.com/news/ev/is-the-global-ev-market-slowing-down/", "https://ev-volumes.com/news/ev/how-have-global-ev-forecasts-reacted-to-strong-headwinds/",
#          "https://ev-volumes.com/news/ev/how-is-europes-ev-slowdown-affecting-its-market-forecast/", "https://ev-volumes.com/news/ev/europes-truck-markets-continue-to-electrify/", "https://ev-volumes.com/news/ev/how-popular-are-electric-buses-in-europe/",
#          "https://ev-volumes.com/news/ev/global-ev-growth-forecast-in-2024-but-challenges-remain/", "https://ev-volumes.com/news/ev/tesla-dominates-european-ev-market-in-february/",
#          "https://ev-volumes.com/news/ev/growth-forecast-for-european-ev-market-despite-incentive-impact/", "https://ev-volumes.com/news/ev/what-was-the-most-popular-ev-worldwide-in-2023/",
#          "https://ev-volumes.com/news/ev/global-ev-sales-for-2023/", "https://ev-volumes.com/news/ev/the-green-transition-of-electric-trucks-and-buses/", "https://ev-volumes.com/news/ev/evs-forecast-to-account-for-two-thirds-of-global-light-vehicle-sales-in-2035/",
#          "https://ev-volumes.com/news/ev/global-ev-sales-for-2023-h1/", "https://ev-volumes.com/news/ev/global-ev-sales-for-2022/"]

In [ ]:
# article_lst=[]

# for url in urls2:
#   article = Article(url)
#   article.download()
#   article.parse()
#   article_lst.append(article.text)

In [ ]:
# df2['article'] = article

In [ ]:
# df2

,date,title,article
0,20241118,On song BYD dominates record-breaking Chinese ...,<newspaper.article.Article object at 0x7fca8c6...
1,20241002,Is Europe’s EV decline impacting market foreca...,<newspaper.article.Article object at 0x7fca8c6...
2,20240919,Which EV manufacturers performed the best in 2...,<newspaper.article.Article object at 0x7fca8c6...
3,20240905,Is the global EV market slowing down?,<newspaper.article.Article object at 0x7fca8c6...
4,20240819,How have global EV forecasts reacted to strong...,<newspaper.article.Article object at 0x7fca8c6...
5,20240725,How is Europe’s EV slowdown affecting its mark...,<newspaper.article.Article object at 0x7fca8c6...
6,20240718,Europe’s medium and heavy-duty truck markets c...,<newspaper.article.Article object at 0x7fca8c6...
7,20240613,How popular are electric buses in Europe?,<newspaper.article.Article object at 0x7fca8c6...
8,20240527,"Global EV growth forecast in 2024, but challen...",<newspaper.article.Article object at 0x7fca8c6...
9,20240422,Tesla dominates European EV market in February,<newspaper.article.Article object at 0x7fca8c6...


## 3. 데이터 저장

In [ ]:
# df1.to_csv("/content/gdrive/MyDrive/통계분석실습/기말프로젝트/data/cox_automotive.csv", index=False)

In [ ]:
# df2.to_csv("/content/gdrive/MyDrive/통계분석실습/기말프로젝트/data/ev_volumes.csv", index=False)

In [ ]:
df3.to_csv('/content/gdrive/MyDrive/통계분석실습/기말프로젝트/data/insiedEV_hyd.csv', index=False)

In [ ]:
df4.to_csv('/content/gdrive/MyDrive/통계분석실습/기말프로젝트/data/insiedEV_kia.csv', index=False)

In [ ]:
df1

,date,title,article
0,20230412,Another Record Broken: Q1 Electric Vehicle Sal...,"For more than a year now, the Industry Insight..."
1,20230612,Electric Vehicle Sales in Q2 Strike Another Re...,"Nearly 300,000 new electric vehicles (EVs) – f..."
2,20231012,"Another Quarter, Another Record: EV Sales in t...",Electric vehicle (EV) sales volumes set anothe...
3,20240109,A Record 1.2 Million EVs Were Sold in the U.S....,A record 1.2 million U.S. vehicle buyers chose...
4,20240411,EV Sales Growth Slows; Market Leader Tesla Stalls,Electric vehicle (EV) sales growth in the U.S....
5,20240611,"Despite Tesla Slide, EV Sales in Q2 Mark New R...",Electric vehicle sales in the U.S. grew by 11....
6,20241011,Electric Vehicle Sales Mark Another Record in ...,According to new estimates from Kelley Blue Bo...


In [ ]:
df2

,date,title,article
0,20241118,On song BYD dominates record-breaking Chinese ...,Chinese electric vehicle (EV) deliveries soare...
1,20241002,Is Europe’s EV decline impacting market foreca...,"With Europe’s light vehicle market slowing, ho..."
2,20240919,Which EV manufacturers performed the best in 2...,"""Which electric vehicle (EV) manufacturers sto..."
3,20240905,Is the global EV market slowing down?,New electric vehicle (EV) sales across the wor...
4,20240819,How have global EV forecasts reacted to strong...,Global electric vehicle (EV) sales are experie...
5,20240725,How is Europe’s EV slowdown affecting its mark...,Europe’s electric vehicle (EV) market has slow...
6,20240718,Europe’s medium and heavy-duty truck markets c...,The electrification of Europe’s medium and hea...
7,20240613,How popular are electric buses in Europe?,Europe’s electric-bus market ended 2023 on a h...
8,20240527,"Global EV growth forecast in 2024, but challen...",Despite current market conditions leading to a...
9,20240422,Tesla dominates European EV market in February,Electric vehicle (EV) registrations in Europe ...


In [ ]:
ev_market = pd.concat([df1, df2], ignore_index=True)
ev_market = ev_market.sort_values(by=['date']).reset_index(drop=True)

In [ ]:
ev_market

,date,title,article
0,20230206,Global EV Sales for 2022,Global EV sales continued strong. A total of 1...
1,20230412,Another Record Broken: Q1 Electric Vehicle Sal...,"For more than a year now, the Industry Insight..."
2,20230612,Electric Vehicle Sales in Q2 Strike Another Re...,"Nearly 300,000 new electric vehicles (EVs) – f..."
3,20230817,Global EV Sales for 2023 H1,Global EV sales continue strong. A total of 6 ...
4,20231012,"Another Quarter, Another Record: EV Sales in t...",Electric vehicle (EV) sales volumes set anothe...
5,20231121,EVs Forecast to Account for Two Thirds of Glob...,"We forecast that global EV sales, comprising b..."
6,20240109,A Record 1.2 Million EVs Were Sold in the U.S....,A record 1.2 million U.S. vehicle buyers chose...
7,20240122,Global EV Sales for 2023,Global EV sales continued as expected by us at...
8,20240219,What was the most popular EV worldwide in 2023?,The global electric vehicle (EV) market broke ...
9,20240327,Growth forecast for European EV market despite...,"Even with incentive changes, Europe’s new elec..."


In [ ]:
ev_market.to_csv("/content/gdrive/MyDrive/통계분석실습/기말프로젝트/data/ev_market.csv", index=False)